<a href="https://colab.research.google.com/github/jcmachicao/MachineLearningAvanzado_UC_2025/blob/main/U2__NN_Simple_Keras_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ejemplo simple de red neuronal en Keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Parámetros del problema
n_eventos = 1000   # número de filas (ejemplos)
n_columnas = 20    # número de columnas (features)

# Datos de ejemplo aleatorios (X) y etiquetas (y)
X = np.random.rand(n_eventos, n_columnas).astype(np.float32)
y = np.random.randint(0, 2, size=(n_eventos, 1))  # salida binaria (0/1)

# Definición del modelo
model = keras.Sequential([
    layers.Input(shape=(n_columnas,)),       # entrada: columnas por evento
    layers.Dense(100, activation='relu'),    # capa 1
    layers.Dense(100, activation='relu'),    # capa 2
    layers.Dense(100, activation='relu'),    # capa 3
    layers.Dense(1, activation='sigmoid')    # salida unitaria (binaria)
])

# Compilación del modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Entrenamiento
history = model.fit(
    X, y,
    epochs=20,        # número de ciclos (épocas)
    batch_size=32,    # tamaño de lote
    validation_split=0.2,  # separación para validación
    verbose=1
)

# Evaluación
loss, acc = model.evaluate(X, y, verbose=0)
print(f"Precisión final sobre todos los datos: {acc:.3f}")


In [ ]:
# Ejemplo simple de red neuronal en PyTorch

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Parámetros del problema
n_eventos = 1000
n_columnas = 20

# Datos aleatorios
X = torch.rand((n_eventos, n_columnas))
y = torch.randint(0, 2, (n_eventos, 1)).float()  # salida binaria

# Dividimos en train y validación
X_train, X_val, y_train, y_val = train_test_split(
    X.numpy(), y.numpy(), test_size=0.2, random_state=42
)

X_train, X_val = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_val, dtype=torch.float32)
y_train, y_val = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)

# Definición del modelo
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 100),  # capa 1
            nn.ReLU(),
            nn.Linear(100, 100),        # capa 2
            nn.ReLU(),
            nn.Linear(100, 100),        # capa 3
            nn.ReLU(),
            nn.Linear(100, 1),          # salida unitaria
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

model = SimpleNN(n_columnas)

# Función de pérdida y optimizador
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
epochs = 20
batch_size = 32

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train.size()[0])

    for i in range(0, X_train.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Validación
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)
        val_acc = ((val_outputs > 0.5) == y_val).float().mean()

    print(f"Época {epoch+1}/{epochs} - Pérdida validación: {val_loss:.4f} - Acc val: {val_acc:.3f}")

